In [215]:
import pandas as pd
import numpy as np
from collections import Counter

In [216]:
data = pd.read_csv('data/soc-sign-bitcoinotc.csv')

```python
filtered_data = data.query('Evaluation >= 8')
filtered_data.to_csv(path_or_buf='data/filtered-sign-bitcoinotc.csv', sep=',', columns=['Source', 'Target', 'Evaluation'])
```

In [217]:
filtered_data = data.query('Evaluation >= 8')
source = list(filtered_data.Source)
target = list(filtered_data.Target)
nodes = list(set(source).union(set(target)))

In [218]:
adjacence_dict = {no:[] for no in nodes}

for index in range(len(source)):
    ori = source[index]
    dest = target[index]
    
    adjacence_dict[ori].append(dest)

In [219]:
a = [[0 for i in range(len(nodes))] for j in range(len(nodes))]

for i in range(len(nodes)):
    for j in range(len(nodes)):
        ori = nodes[i]
        dest = nodes[j]
        out_degree = len(adjacence_dict[ori])
        a[j][i] = 1/out_degree if dest in adjacence_dict[ori] else 0

a = np.matrix(a)

In [220]:
b = (1/len(nodes)) * np.matrix([[1 for i in range(len(nodes))] for j in range(len(nodes))])

In [221]:
m = 0.85 * a + 0.15 * b

In [222]:
v = (1/len(nodes)) * np.matrix([[1] for i in range(len(nodes))])

In [223]:
def pagerank(v):
    if sum(abs(m*v-v)) > 0.001:
        return pagerank(m*v)
    return m*v

In [224]:
result = pagerank(v)
result = [cell.item(0,0) for cell in result]

In [225]:
result_data_frame = pd.DataFrame({'id': nodes, 'PageRank': result})

In [226]:
result_data_frame.to_csv(path_or_buf='results/result.csv', index=False)
filtered_data.to_csv(path_or_buf='results/filtered-sign-bitcoinotc.csv', columns=['Source', 'Target'], index=False)